In [1]:
import matplotlib
matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec

import numpy as np

from utility_match_units import (binary_reader_waveforms,match_units,

                                 search_spikes_parallel, load_ks2_spikes,
                     reload_ks2_templates, Match_to_ground_truth,
                     plot_single_unit)

clrs = ['blue', 'red', 'green', 'cyan',
'black','grey','brown','slategrey','darkviolet','darkmagenta',
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']


Autosaving every 180 seconds


In [2]:
# dataset aprameters:
root_dir = '/media/cat/12TB/insync_cm3746/paninski_lab/data/neuropixels/synthetic_drift/run15_30mins_40umDrift/'
n_channels = 384
n_times = 101

# convert KS2 spike trains to 2 column standard and reorder spikes by sequential index
if True:
    spike_train_sorted = load_ks2_spikes(root_dir, n_channels, n_times)
#print ("Spike train: \n", spike_train_sorted)
else:
    spike_train_sorted = np.load(root_dir+'tmp/output/spike_train.npy')
    spike_train_sorted[:,0]-=n_times//2
    
# remake templates from data
print ("Remaking templates ")
data_type='int16'
templates_sorted = reload_ks2_templates(root_dir, spike_train_sorted, data_type, 
                                        n_channels, n_times)
print ("templates sorted: ", templates_sorted.shape)


Remaking templates 
templates sorted:  (879, 101, 384)


In [3]:
# plt.plot(templates_sorted[0])
# plt.show()

In [4]:
# match to ground truth data;
matcher = Match_to_ground_truth(root_dir, spike_train_sorted, templates_sorted)
print ("n_spikes: ", matcher.n_spikes)
print (" DONE MATCHING ")
matcher.n_times = n_times
matcher.n_channels = n_channels

 ground truth templates:  (100, 101, 384)
units_ground_truth:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16]
units_ground_truth:  [17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33]
units_ground_truth:  [34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50]
units_ground_truth:  [51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67]
searching match unit:  0
# spikes in sorted unit:  180
# matches:  1
searching match unit:  1
# spikes in sorted unit:  157
# matches:  0
searching match unit:  2
# spikes in sorted unit:  198
# matches:  0
searching match unit:  3
# spikes in sorted unit:  275
# matches:  3
searching match unit:  4
# spikes in sorted unit:  369
# matches:  4
searching match unit:  5
# spikes in sorted unit:  229
# matches:  3
searching match unit:  6
# spikes in sorted unit:  181
# matches:  1
searching match unit:  7
# spikes in sorted unit:  182
# matches:  5
units_ground_truth:  [68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83]
units_ground_truth:  [84 85 86 87 88 89

# matches:  2
searching match unit:  110
# spikes in sorted unit:  180
# matches:  3
searching match unit:  111
# spikes in sorted unit:  382
# matches:  2
searching match unit:  112
# spikes in sorted unit:  185
# matches:  1
searching match unit:  113
# spikes in sorted unit:  11147
# matches:  101
searching match unit:  114
# spikes in sorted unit:  185
# matches:  3
searching match unit:  115
# spikes in sorted unit:  15654
# matches:  135
searching match unit:  116
# spikes in sorted unit:  180
# matches:  2
searching match unit:  117
# spikes in sorted unit:  180
# matches:  5
searching match unit:  118
# spikes in sorted unit:  381
# matches:  3
searching match unit:  119
# spikes in sorted unit:  164
# matches:  6
searching match unit:  120
# spikes in sorted unit:  13876
# matches:  138
searching match unit:  121
# spikes in sorted unit:  181
# matches:  0
searching match unit:  122
# spikes in sorted unit:  180
# matches:  2
searching match unit:  123
# spikes in sorted unit:

# matches:  76
searching match unit:  225
# spikes in sorted unit:  180
# matches:  2
searching match unit:  226
# spikes in sorted unit:  179
# matches:  3
searching match unit:  227
# spikes in sorted unit:  372
# matches:  3
searching match unit:  228
# spikes in sorted unit:  184
# matches:  1
searching match unit:  229
# spikes in sorted unit:  337
# matches:  3
searching match unit:  230
# spikes in sorted unit:  12232
# matches:  98
searching match unit:  231
# spikes in sorted unit:  197
# matches:  1
searching match unit:  232
# spikes in sorted unit:  180
# matches:  2
searching match unit:  233
# spikes in sorted unit:  966
# matches:  15
searching match unit:  234
# spikes in sorted unit:  338
# matches:  3
searching match unit:  235
# spikes in sorted unit:  180
# matches:  1
searching match unit:  236
# spikes in sorted unit:  192
# matches:  4
searching match unit:  237
# spikes in sorted unit:  185
# matches:  4
searching match unit:  238
# spikes in sorted unit:  180
#

# spikes in sorted unit:  247
# matches:  3
searching match unit:  340
# spikes in sorted unit:  14895
# matches:  137
searching match unit:  341
# spikes in sorted unit:  181
# matches:  2
searching match unit:  342
# spikes in sorted unit:  186
# matches:  2
searching match unit:  343
# spikes in sorted unit:  208
# matches:  5
searching match unit:  344
# spikes in sorted unit:  181
# matches:  4
searching match unit:  345
# spikes in sorted unit:  191
# matches:  0
searching match unit:  346
# spikes in sorted unit:  185
# matches:  2
searching match unit:  347
# spikes in sorted unit:  364
# matches:  4
searching match unit:  348
# spikes in sorted unit:  151
# matches:  4
searching match unit:  349
# spikes in sorted unit:  179
# matches:  2
searching match unit:  350
# spikes in sorted unit:  304
# matches:  3
searching match unit:  351
# spikes in sorted unit:  18568
# matches:  156
searching match unit:  352
# spikes in sorted unit:  226
# matches:  0
searching match unit:  35

# matches:  73
searching match unit:  454
# spikes in sorted unit:  151
# matches:  2
searching match unit:  455
# spikes in sorted unit:  285
# matches:  3
searching match unit:  456
# spikes in sorted unit:  185
# matches:  1
searching match unit:  457
# spikes in sorted unit:  8009
# matches:  74
searching match unit:  458
# spikes in sorted unit:  185
# matches:  0
searching match unit:  459
# spikes in sorted unit:  17365
# matches:  170
searching match unit:  460
# spikes in sorted unit:  540
# matches:  5
searching match unit:  461
# spikes in sorted unit:  182
# matches:  1
searching match unit:  462
# spikes in sorted unit:  11211
# matches:  100
searching match unit:  463
# spikes in sorted unit:  233
# matches:  2
searching match unit:  464
# spikes in sorted unit:  14329
# matches:  141
searching match unit:  465
# spikes in sorted unit:  360
# matches:  6
searching match unit:  466
# spikes in sorted unit:  180
# matches:  0
searching match unit:  467
# spikes in sorted un

# spikes in sorted unit:  188
# matches:  4
searching match unit:  569
# spikes in sorted unit:  180
# matches:  2
searching match unit:  570
# spikes in sorted unit:  19831
# matches:  187
searching match unit:  571
# spikes in sorted unit:  184
# matches:  5
searching match unit:  572
# spikes in sorted unit:  187
# matches:  1
searching match unit:  573
# spikes in sorted unit:  203
# matches:  3
searching match unit:  574
# spikes in sorted unit:  189
# matches:  3
searching match unit:  575
# spikes in sorted unit:  9840
# matches:  69
searching match unit:  576
# spikes in sorted unit:  5337
# matches:  48
searching match unit:  577
# spikes in sorted unit:  181
# matches:  1
searching match unit:  578
# spikes in sorted unit:  199
# matches:  6
searching match unit:  579
# spikes in sorted unit:  184
# matches:  0
searching match unit:  580
# spikes in sorted unit:  1785
# matches:  13
searching match unit:  581
# spikes in sorted unit:  181
# matches:  0
searching match unit:  

# spikes in sorted unit:  4065
# matches:  39
searching match unit:  684
# spikes in sorted unit:  231
# matches:  1
searching match unit:  685
# spikes in sorted unit:  342
# matches:  2
searching match unit:  686
# spikes in sorted unit:  5638
# matches:  58
searching match unit:  687
# spikes in sorted unit:  4887
# matches:  49
searching match unit:  688
# spikes in sorted unit:  181
# matches:  2
searching match unit:  689
# spikes in sorted unit:  161
# matches:  2
searching match unit:  690
# spikes in sorted unit:  447
# matches:  2
searching match unit:  691
# spikes in sorted unit:  10018
# matches:  93
searching match unit:  692
# spikes in sorted unit:  3685
# matches:  52
searching match unit:  693
# spikes in sorted unit:  15547
# matches:  130
searching match unit:  694
# spikes in sorted unit:  173
# matches:  5
searching match unit:  695
# spikes in sorted unit:  5367
# matches:  39
searching match unit:  696
# spikes in sorted unit:  226
# matches:  2
searching match 

# matches:  4
searching match unit:  798
# spikes in sorted unit:  332
# matches:  5
searching match unit:  799
# spikes in sorted unit:  3198
# matches:  24
searching match unit:  800
# spikes in sorted unit:  214
# matches:  6
searching match unit:  801
# spikes in sorted unit:  310
# matches:  1
searching match unit:  802
# spikes in sorted unit:  268
# matches:  2
searching match unit:  803
# spikes in sorted unit:  209
# matches:  2
searching match unit:  804
# spikes in sorted unit:  662
# matches:  3
searching match unit:  805
# spikes in sorted unit:  429
# matches:  5
searching match unit:  806
# spikes in sorted unit:  1308
# matches:  7
searching match unit:  807
# spikes in sorted unit:  2002
# matches:  14
searching match unit:  808
# spikes in sorted unit:  456
# matches:  2
searching match unit:  809
# spikes in sorted unit:  376
# matches:  5
searching match unit:  810
# spikes in sorted unit:  175
# matches:  3
searching match unit:  811
# spikes in sorted unit:  179
#

In [5]:
# Make Pie Charts from results;
if True:
    n_matches = 3
    matcher.make_pie_charts(n_matches)
    

n matches:  3
sizes: [16339   368   295]  , clrs:  ['blue', 'red', 'green']
sizes: [7158  435  367]  , clrs:  ['blue', 'red', 'green']
sizes: [10709   228   220]  , clrs:  ['blue', 'red', 'green']
sizes: [6820  403  338]  , clrs:  ['blue', 'red', 'green']
sizes: [5663  123  117]  , clrs:  ['blue', 'red', 'green']
sizes: [3647   83   77]  , clrs:  ['blue', 'red', 'green']
sizes: [15262   371   319]  , clrs:  ['blue', 'red', 'green']
sizes: [13852   315   271]  , clrs:  ['blue', 'red', 'green']
sizes: [11352   261   193]  , clrs:  ['blue', 'red', 'green']
sizes: [3134   78   68]  , clrs:  ['blue', 'red', 'green']
sizes: [16637   383   312]  , clrs:  ['blue', 'red', 'green']
sizes: [13516   305   258]  , clrs:  ['blue', 'red', 'green']
sizes: [12086   287   228]  , clrs:  ['blue', 'red', 'green']
sizes: [6288  145  117]  , clrs:  ['blue', 'red', 'green']
sizes: [16281   363   295]  , clrs:  ['blue', 'red', 'green']
sizes: [16463   548   406]  , clrs:  ['blue', 'red', 'green']
sizes: [9155

In [6]:
scale_amplitude = 10.
n_matches = 3
save_fig = True
# plot ptp of spikes from different units:
# selected_unit = 15 #15 #47 #39 #21

units = np.unique(matcher.spike_train_gt[:,1])
for selected_unit in units: 
    plot_single_unit(root_dir, selected_unit, matcher, 
                  n_matches, clrs, scale_amplitude,
                  save_fig)

matched_units:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 24


 unit:  2 , matching unit:  637 637

 unit:  2 , matching unit:  255 255

 loading injected unit:  2
 ground truth wfs:  (10738, 101, 384)
len ptpssorted:  3
rec len sec:  1800
matched_units:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 19


 unit:  5 , matching unit:  637 637

 unit:  5 , matching unit:  518 518

 loading injected unit:  5
 ground truth wfs:  (3756, 101, 384)
len ptpssorted:  3
rec len sec:  1800
matched_units:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198


 unit:  8 , matching unit:  637 637

 unit:  8 , matching unit:  406 406

 loading injected unit:  8
 ground truth wfs:  (11368, 101, 384)
len ptpssorted:  3
rec len sec:  1800
matched_units:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 19


 unit:  11 , matching unit:  637 637

 unit:  11 , matching unit:  255 255

 loading injected unit:  11
 ground truth wfs:  (13533, 101, 384)
len ptpssorted:  3
rec len sec:  1800
matched_units:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197


...


 unit:  86 , matching unit:  255 255

 unit:  86 , matching unit:  637 637

 loading injected unit:  86
 ground truth wfs:  (3185, 101, 384)
len ptpssorted:  3
rec len sec:  1800
matched_units:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 


 unit:  89 , matching unit:  637 637

 unit:  89 , matching unit:  255 255

 loading injected unit:  89
 ground truth wfs:  (11929, 101, 384)
len ptpssorted:  3
rec len sec:  1800
matched_units:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197



 unit:  92 , matching unit:  637 637

 unit:  92 , matching unit:  255 255

 loading injected unit:  92
 ground truth wfs:  (13472, 101, 384)
len ptpssorted:  3
rec len sec:  1800
matched_units:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197



 unit:  95 , matching unit:  637 637

 unit:  95 , matching unit:  612 612

 loading injected unit:  95
 ground truth wfs:  (5048, 101, 384)
len ptpssorted:  3
rec len sec:  1800
matched_units:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 


 unit:  98 , matching unit:  637 637

 unit:  98 , matching unit:  612 612

 loading injected unit:  98
 ground truth wfs:  (17596, 101, 384)
len ptpssorted:  3
rec len sec:  1800
matched_units:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197


In [ ]:
print (len(ptp_gt_unit), len(purity),len(completeness))
print (ptp_gt_unit)
print (purity)
ax=plt.subplot(121)
plt.scatter(ptp_gt_unit, purity, c='blue')
plt.title("True Positive Rate \n(# spikes found / # spikes injected)", fontsize=15)
plt.xlabel("PTP of unit", fontsize=20)
plt.xlim(left=0)
plt.ylim(0,1.05)
ax.tick_params(axis='both', which='major', labelsize=20)

# plot purity
ax=plt.subplot(122)
ax.tick_params(axis='both', which='major', labelsize=20)
plt.scatter(ptp_gt_unit,completeness, c='red')
plt.title("Purity \n(# spikes found / # spikes in unit)", fontsize=15)
plt.xlabel("PTP of injected unit", fontsize=20)
plt.xlim(left=0)
plt.ylim(0,1.05)
plt.suptitle("Neuropixels Drift Simulation run3\n shifts: "+
             str(np.array([-0.1,  -0.05,  0.,    0.05])*40)+" um"+ " @ "+
             str(np.array([0.,  6000000., 12000000., 18000000.])/30000)+ " sec"+
            "\n10mins recording, 384 channels"+
            "\n Sorted using KS2")
plt.show()

In [ ]:
t1 = np.load('/media/cat/1TB/data/synthetic/run3/ground_truth/templates_ground_truth.npy')
print (t1.shape)
t2 = np.load('/media/cat/1TB/data/synthetic/run3/templates_reloaded_good.npy')
print (t2.shape)

max1 = t1[2].ptp(0).argmax(0)
max2 = t2[50].ptp(0).argmax(0)
print (max1, max2)

ax=plt.subplot(121)
plt.plot(t1[2],c='black')
ax=plt.subplot(122)
plt.plot(t2[50]/10.,c='blue')
plt.show()


In [ ]:
d1 = max_chans_sorted
d2 = max_chans_unit
t1, t2, t3 = np.intersect1d(d1,d2,return_indices=True)
print (t1,t2,t3)

t1 = np.where(np.in1d(d1, d2))[0]
print (t1)



In [67]:
spike_train = np.load('/media/cat/1TB/data/synthetic/run3/ground_truth/spike_train_ground_truth.npy')
print (spike_train)
templates = np.load('/media/cat/1TB/data/synthetic/run3/ground_truth/templates_ground_truth.npy')
print (templates.shape)

units = np.load('/media/cat/1TB/data/synthetic/run3/ground_truth/units.npy')
print (units)

ctr=0
spike_train_corrected = spike_train.copy()
for unit in units:
    idx = np.where(spike_train[:,1]==unit)[0]
    spike_train_corrected[idx,1]=ctr
    ctr+=1
    
print (spike_train_corrected)
np.save('/media/cat/1TB/data/synthetic/run3/ground_truth/spike_train_ground_truth_corrected.npy',spike_train_corrected)

[[      35      420]
 [    6119      420]
 [   12204      420]
 ...
 [17983395      247]
 [17990820      247]
 [17998245      247]]
(49, 101, 384)
[420 339 421 348 473 418  62 362 565 239 498 474 324 298 471 441 417 443
 356 424  63 437 485 160 156 302 346 334 431 486  66 204  14 258 300  16
 150  52 317 277 184 456  65 481  38 436 558 194 247]
[[      35        0]
 [    6119        0]
 [   12204        0]
 ...
 [17983395       48]
 [17990820       48]
 [17998245       48]]


In [21]:
units=np.arange(5)
units = [good_units[3]]

for unit in units:
    fig = plt.figure()
    ax = plt.subplot(131)
    #plt.plot(templates[unit].T)
    plt.plot(wfs_array[unit].mean(0))

    ax = plt.subplot(132)
    #spikes = spikes[idx]
    spikes = times[unit]
    print ("spikes:", spikes.shape, "ptps: ", ptps[unit].shape)
    plt.scatter(spikes/30000., ptps[unit])

    ax=plt.subplot(133)
    max_chan = wfs_array[unit].mean(0).ptp(0).argmax(0)#templates[unit].ptp(0).argmax(0)
    print (wfs_array[unit].shape)

    cmap = cm.get_cmap('viridis',wfs_array[unit].shape[0])
    clrs = cmap(np.arange(wfs_array[unit].shape[0]))
    print (clrs)
    for k in range(0,wfs_array[unit].shape[0],10):
        plt.plot(wfs_array[unit][k].T,c=cmap(k))


plt.show()



spikes: (8210,) ptps:  (8210,)
(8210, 101)
[[0.267004 0.004874 0.329415 1.      ]
 [0.267004 0.004874 0.329415 1.      ]
 [0.267004 0.004874 0.329415 1.      ]
 ...
 [0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]
 [0.993248 0.906157 0.143936 1.      ]]


In [9]:
fname = '/home/cat/Downloads/yass/samples/10chan/phy/whitening_mat.npy'

n_channels = 10

whitening_mat = np.random.rand(n_channels,n_channels)

np.save(fname, whitening_mat)
print (whitening_mat.shape)
print (whitening_mat)

(10, 10)
[[0.61659694 0.20125714 0.48515898 0.45059603 0.89409843 0.8158124
  0.30200264 0.06808628 0.60955651 0.7760584 ]
 [0.19814057 0.21134196 0.27791917 0.52312241 0.20263871 0.21097603
  0.34816364 0.1852993  0.90506002 0.80562087]
 [0.970196   0.70466193 0.08746122 0.33371635 0.44791585 0.00678562
  0.41421277 0.65443008 0.8541354  0.41522301]
 [0.88052786 0.34072116 0.12298287 0.23288985 0.98943396 0.56546028
  0.38736253 0.87851743 0.83882004 0.96018176]
 [0.5622632  0.19456103 0.89831444 0.56587043 0.12599613 0.24817026
  0.47188959 0.55809954 0.72628178 0.72821972]
 [0.59236306 0.06332327 0.35662913 0.93425548 0.15622572 0.77164442
  0.62013196 0.67339586 0.97380542 0.9211003 ]
 [0.55011567 0.22242238 0.41892959 0.23396425 0.80036351 0.00612786
  0.72244072 0.1119026  0.84247935 0.13666388]
 [0.1674969  0.17014984 0.70297569 0.28534628 0.87704159 0.62894665
  0.38268201 0.47207143 0.53495952 0.46683818]
 [0.58624526 0.67612232 0.60880662 0.66169478 0.10866143 0.88197916
  0.

In [24]:
max_chans = templates.ptp(1).argmax(1)
ptps = templates.ptp(1).max(1)
ptps_all = templates.ptp(1)
print (ptps_all.shape)

print (max_chans)
print (ptps)


(426, 364)
[  0   1   1   1   2   3   4   6   7   5  10  13  14  14  15  16  17  18
  20  21  30  26  29  29  30  31  32  35  37  41  42  43  45  46  49  50
  38  56  58  60  69  68  68  68  72  70  71  74  75  76 342  80  79  80
  80  83  82  84 333  85  86  87  88  88  88  88  89  89  90  88  87  92
 336  94  95  98  99  84 101 103 104 108 109 109 111 111 111 112 114 115
 119 119 120 121 125 127 127 124 131 133 134 134 132 140 137 138 140 332
 140 140 141 142 142 140 143 145 146 147 147 149 149 149 149 150 284 149
 154 154 154 162 164 165 166 164 169 170 173 173 174 177 178 173 180 181
 182 186 182 185 185 186 186 189 190 192 194 197 198 199 201 204 204 206
 208 208 209 209 213 209 210 212 205 209 213 213 213 213 216 216 216 217
 219 220 226 228 229 229 229 230 230 233 235 215 236 236 238 239 239 242
 245 245 248 248 248 248 249 249 252 252 253 253 262 262 262 262 262 262
 262 262 262 262 262 262 252 265 265 265 266 266 257 268 268 253 273 273
 274 277 277 277 280 262 280 282 280 284

In [33]:
# plot scatter vs. depth
fig=plt.figure()
ax=plt.subplot(111)
for k in range(426):
    idx = np.where(spike_clusters==k)[0]
    times = spike_times[idx]/30000.
    plt.scatter(times, times*0+geom[max_chans[k],1]+(np.random.rand()*80-40),s=1, alpha=.9)
    
ax.tick_params(axis = 'both', which = 'major', labelsize = 20)
plt.ylim(3800,-100)
plt.xlim(-10,times[-1]+10)
plt.ylabel("Depth (um)",fontsize=20)
plt.xlabel("Time (sec)",fontsize=20)

plt.show()



In [59]:
import wavefile

data = wavefile.loadWave('/home/cat/Downloads/time code signal.wav')

print (data[1].shape)

plt.plot(data[1].squeeze()[:1000])
plt.show()

(1, 200000)


In [70]:
datas = data[1].squeeze()
print (datas.shape)
print (datas)

(200000,)
[ 0.5  0.5  0.5 ... -0.5 -0.5 -0.5]


In [74]:
idx = np.where(datas>0)[0]
print (idx)
diffs = idx[1:]-idx[:-1]
print (diffs)

[     0      1      2 ... 199992 199993 199994]
[1 1 1 ... 1 1 1]


In [75]:
start = []
end = []
start.append(0)
for k in range(1, diffs.shape[0]-1, 1):
    if (diffs[k]-diffs[k-1])>1:
        end.append(k-1)
        start.append(k)
end.append(k)
end = np.array(end)
start = np.array(start)
print ((end-start)[:100])
diffs_redone = end-start+1
print (diffs_redone)

[ 7 10  8  9  9  9  9  9  9  9  9  9  9  9  4  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  9  4  4  4  4  4  4  4  4  4  4  4  4  4  4
  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  4  4  4  4  4  4  4  4  4  4  4  4  4  9  4
  9  9  9  9]
[ 8 11  9 ...  5  5  4]


In [79]:
ax=plt.subplot(111)
sample_rate = 20000.
hist = np.histogram(diffs_redone, np.arange(-20,50,1))
plt.bar(hist[1][:-1]/sample_rate,hist[0],0.9/sample_rate)
plt.xlabel("Inter pulse interval (seconds)")
plt.title("Distribution of inter-spike-interval (20 sec wav file provided by Avisoft)")
#ax.set_yscale('log')
plt.show()